In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date, Irr, chart_dates, chart_labels, chart_colour, portfolio_at_date, calculate_security_nav
from datetime import date

In [2]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'

In [3]:
from datetime import timedelta
currency_target = 'EUR'

security_id = 7
target_date = date(2021, 1, 8)
broker_id_list = None
start_date = None

security = Assets.objects.get(id=security_id)
is_long_position = None

# print(security.transactions.all())

transactions = security.transactions.filter(
    quantity__isnull=False,  # Filter out transactions where quantity is not empty
    date__lte=target_date
).values('price', 'quantity', 'date', 'currency')

if broker_id_list is not None:
    transactions = transactions.filter(broker_id__in=broker_id_list) 

print(transactions.order_by('-date').first()['date'])

# Step 2: Calculate based on the period starting from the previous position == 0 or start_date, whichever is later
if security.position(target_date) == 0:
    latest_zero_position = security.dates_of_zero_positions(target_date)[1]
else:
    latest_zero_position = security.dates_of_zero_positions(target_date)[0]

print('Entry date', latest_zero_position)

if start_date and start_date > latest_zero_position:
    latest_zero_position = start_date

    transactions = transactions.filter(date__gt=latest_zero_position)
    position = security.position(latest_zero_position, broker_id_list)
    if position != 0:
        transactions = list(transactions) + [{
            'price': security.price_at_date(latest_zero_position).price,
            'quantity': position,
            'date': latest_zero_position,
            'currency': security.currency,
        }]
        is_long_position = position > 0
else:
    transactions = transactions.filter(date__gt=latest_zero_position)

if is_long_position is None:
    first_transaction = transactions.order_by('date').first()
    is_long_position = first_transaction['quantity'] > 0

print(transactions)

# Step 3: Amend the calculation method. For every entry transaction buy-in price is the weighted average of previous buy-in price and price for the current transaction.
value = 0
quantity = 0
for transaction in transactions:
    fx_rate = FX.get_rate(transaction['currency'], currency_target, transaction['date'])['FX']
    if fx_rate:
        previous_buy_in_price = value / quantity if quantity else 0
        current_price = transaction['price'] * fx_rate
        weight_previous = quantity
        weight_current = transaction['quantity']
        # If it's a long position and the quantity is positive, or if it's a short position and the quantity is negative, use the current price. Otherwise, use the previous buy-in price.
        price = current_price if (is_long_position and transaction['quantity'] > 0) or (not is_long_position and transaction['quantity'] < 0) else previous_buy_in_price
        print(previous_buy_in_price, price, weight_previous, weight_current)
        if (weight_previous + weight_current) == 0:
            buy_in_price = previous_buy_in_price
        else:
            buy_in_price = (previous_buy_in_price * weight_previous + price * weight_current) / (weight_previous + weight_current)
        value = buy_in_price * (quantity + transaction['quantity'])
        quantity += transaction['quantity']

bp = value / quantity if quantity else previous_buy_in_price

print(bp)
# print(security.dates_of_zero_positions(target_date))

2021-01-08
Entry date 2021-01-06
<QuerySet [{'price': Decimal('42.00'), 'quantity': Decimal('-5.00'), 'date': datetime.date(2021, 1, 7), 'currency': 'EUR'}, {'price': Decimal('37.00'), 'quantity': Decimal('2.00'), 'date': datetime.date(2021, 1, 8), 'currency': 'EUR'}]>
0 42.00 0 -5.00
42.00 42.00 -5.00 2.00
42.00


In [12]:
def realized_gain_loss(security, date, currency, broker_id_list=None):
    
    realized_gain_loss = 0
    total_gl_before_entry = 0

    # Step 1: Find the latest date when position is 0
    zero_positions_dates = security.dates_of_zero_positions(date)
    if zero_positions_dates:
        latest_zero_position = security.dates_of_zero_positions(date)[0]
        # print('Latest zero position', latest_zero_position)
    else:
        return 0

    # Step 2: Sum up values of all transactions before that date
    transactions_before_entry = security.transactions.filter(date__lte=latest_zero_position)
    # print("Transactions before current entry date", transactions_before_entry)
    if broker_id_list is not None:
        transactions_before_entry = transactions_before_entry.filter(broker_id__in=broker_id_list)
    
    for transaction in transactions_before_entry:
        fx_rate = FX.get_rate(transaction.currency, currency, transaction.date)['FX']
        if fx_rate:
            total_gl_before_entry -= transaction.price * transaction.quantity * fx_rate
    # print("Total GL before entry", total_gl_before_entry)

    # Step 3: Determine whether it is a long or short position
    position_at_date = security.position(date)

    if position_at_date != 0:
        is_long_position = position_at_date > 0
        exit_type = 'Sell' if is_long_position else 'Buy'
        # print(f"Position at date: {position_at_date}. Exit type: {exit_type}")

        # Step 4: Calculate realized gain/loss based on exit price and buy-in price
        exit_transactions = security.transactions.filter(type=exit_type, date__gte=latest_zero_position, date__lte=date)
        # print("Exit transactions", exit_transactions)

        for exit in exit_transactions:
            buy_in_price = security.calculate_buy_in_price(exit.date, exit.currency, broker_id_list)
            # print("Buy-in price", buy_in_price)
            if buy_in_price is not None:
                fx_rate = FX.get_rate(exit.currency, currency, exit.date)['FX']
                if fx_rate:
                    realized_gain_loss -= (exit.price * fx_rate - buy_in_price) * (exit.quantity)
                    # print("Realized gain/loss", realized_gain_loss)

    return round(total_gl_before_entry + realized_gain_loss, 2)

target_date = date(2021, 1, 8)
security = Assets.objects.get(id=6)

start_date = date(2020, 12, 31)
end_date = date(2021, 1, 12)
delta = timedelta(days=1)

while start_date <= end_date:
    print(start_date, realized_gain_loss(security, start_date, currency_target, broker_id_list))
    start_date += delta
# print(realized_gain_loss(security, target_date, currency_target, broker_id_list))
# security.dates_of_zero_positions(target_date)

2020-12-31 50.00
2021-01-01 50.00
2021-01-02 50.00
2021-01-03 50.00
2021-01-04 50.00
2021-01-05 50.00
2021-01-06 50.00
2021-01-07 50.00
2021-01-08 50.00
2021-01-09 50.00
2021-01-10 50.00
2021-01-11 50.00
2021-01-12 50.00


In [13]:
def dates_of_zero_positions(security, date, broker_id_list=None):
        """
        Calculate the entry dates for a specific position as of a given date.

        Parameters:
            date (datetime): The date for which the entry dates are calculated.
            broker_id_list (list, optional): A list of broker IDs to filter the transactions. Defaults to None.

        Returns:
            list: A list of entry dates for the position in reversed chronological order.
        """
        
        position = security.position(date, broker_id_list)

        # Get the transactions for this asset
        transactions = security.transactions.filter(date__lte=date, quantity__isnull=False).all()
        if broker_id_list is not None:
            transactions = transactions.filter(broker_id__in=broker_id_list)

        # Order the transactions by date in ascending order
        transactions = transactions.order_by('-date')

        # Initialize the list of entry dates
        entry_dates = []

        # Check if the position is 0
        if position == 0:
            # Add the date of the last transaction to the list of entry dates
            entry_dates.append(transactions.first().date)

        # Iterate over the transactions
        for transaction in transactions:
            # Check if the quantity is not None before subtracting it from the position
            position -= transaction.quantity
            
            # Check if the position is 0
            if position == 0:
                # Add the date of this transaction to the list of entry dates
                entry_dates.append(transaction.date - timedelta(days=1))

        return entry_dates

target_date = date(2021, 1, 7)
dates_of_zero_positions(security, target_date, broker_id_list)

[datetime.date(2020, 6, 30)]

In [29]:
from django.db.models import Sum

def unrealized_gain_loss(security, date, currency, broker_id_list=None):
    """
    Calculate the unrealized gain/loss for this asset.
    Unrealized gain/loss is the difference between the current price of the asset and the purchase price.
    """
    unrealized_gain_loss = 0

    current_position = security.position(date, broker_id_list)
    print(current_position)
    current_price = security.price_at_date(date).price if security.price_at_date(date) else 0
    print(current_price)
    
    buy_in_price = security.calculate_buy_in_price(date, currency, broker_id_list)
    # print("Buy-in price", buy_in_price)
    if buy_in_price is not None:
        fx_rate = FX.get_rate(security.currency, currency, date)['FX']
        if fx_rate:
            unrealized_gain_loss += (current_price * fx_rate - buy_in_price) * (current_position)
    
    return round(unrealized_gain_loss, 2)

target_date = date(2020, 1, 8)
security = Assets.objects.get(id=7)

print(unrealized_gain_loss(security, target_date, currency_target))

start_date = date(2020, 12, 31)
end_date = date(2021, 1, 12)
delta = timedelta(days=1)

while start_date <= end_date:
    print(start_date, unrealized_gain_loss(security, start_date, currency_target))
    start_date += delta

0
0
Error: list index out of range
0
0
20.00
Error: list index out of range
2020-12-31 0
5
20.00
2021-01-01 -25.00
15
20.00
2021-01-02 -125.00
11
20.00
2021-01-03 -91.67
3
20.00
2021-01-04 -25.00
5
20.00
2021-01-05 -55.00
0
20.00
2021-01-06 0.00
-5
20.00
2021-01-07 110.00
-3
20.00
2021-01-08 66.00
0
20.00
2021-01-09 0.00
0
20.00
2021-01-10 0.00
0
20.00
2021-01-11 0.00
0
20.00
2021-01-12 0.00
